<a href="https://colab.research.google.com/github/Kushagratandon12/Machine-Learning-Projects/blob/master/Classification_Happy_%26_Sad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

local_zip = '/tmp/happy-or-sad.zip'
zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

--2020-06-14 07:39:42--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.128, 2404:6800:4008:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.04s   

2020-06-14 07:39:42 (60.0 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [ ]:
train_sad_dir = os.path.join("/tmp/h-or-s/sad")
train_happy_dir = os.path.join("/tmp/h-or-s/happy")
cls_dict = {'0':'sad','1':'happy'}

In [ ]:
#Now Let's List The Data Present 
#Sad Images Directory - open and save in sad_images
sad_images = os.listdir(train_sad_dir)
print(sad_images[:10])
#Happy Images Directory - open and save in happy_images
happy_images =os.listdir(train_happy_dir)
print(happy_images[:10])

['sad2-03.png', 'sad1-12.png', 'sad1-00.png', 'sad2-04.png', 'sad2-06.png', 'sad2-07.png', 'sad2-00.png', 'sad1-11.png', 'sad1-15.png', 'sad1-06.png']
['happy1-00.png', 'happy2-11.png', 'happy1-12.png', 'happy1-01.png', 'happy1-13.png', 'happy2-03.png', 'happy2-02.png', 'happy1-07.png', 'happy1-11.png', 'happy2-01.png']


In [ ]:
print('Sad Images Length {} '.format(len(sad_images)))
print('Happy Images Length ' + str(len(happy_images)))


Sad Images Length 40 
Happy Images Length 40


In [ ]:
#Import Tensorflow - Starting Code
import tensorflow as tf
from sklearn.utils import shuffle

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(8, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(8, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(32, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='softmax') # claculating probability between - o and 1 
])

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 148, 148, 8)       224       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 74, 74, 8)         0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 72, 72, 8)         584       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 36, 36, 8)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 34, 34, 16)        1168      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 17, 17, 16)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 4624)             

In [ ]:
#We Can Use the Adamoptimizer also but where adam optimizers adjust learning rate by itself we can add self learning rate in RMSprop
from tensorflow.keras.optimizers import RMSprop ,Adam

model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=10,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')
print(train_generator.classes)

Found 80 images belonging to 2 classes.
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1]


In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.99):
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True
            
#calling the class 
callbacks=myCallback()

In [ ]:
history = model.fit(
      train_generator,  
      steps_per_epoch=8,
      epochs=20,
      verbose=1,callbacks=[callbacks])

Epoch 1/20
8/8 [==============================] - 0s 18ms/step - loss: 0.1241 - accuracy: 0.9500
Epoch 2/20
8/8 [==============================] - 0s 21ms/step - loss: 0.1156 - accuracy: 0.9500
Epoch 3/20
8/8 [==============================] - 0s 19ms/step - loss: 0.1084 - accuracy: 0.9500
Epoch 4/20
8/8 [==============================] - 0s 18ms/step - loss: 0.1013 - accuracy: 0.9500
Epoch 5/20
8/8 [==============================] - 0s 19ms/step - loss: 0.0975 - accuracy: 0.9625
Epoch 6/20
8/8 [==============================] - 0s 18ms/step - loss: 0.0922 - accuracy: 0.9625
Epoch 7/20
8/8 [==============================] - 0s 19ms/step - loss: 0.0874 - accuracy: 0.9625
Epoch 8/20
8/8 [==============================] - 0s 19ms/step - loss: 0.0852 - accuracy: 0.9500
Epoch 9/20
8/8 [==============================] - 0s 19ms/step - loss: 0.0808 - accuracy: 0.9625
Epoch 10/20
8/8 [==============================] - 0s 19ms/step - loss: 0.0803 - accuracy: 0.9750
Epoch 11/20
8/8 [============